# <center> Typical text preprocessing  
# <center> and architectures comparison example

### <center> Glazunov A.V.

Works via Google Colab.

In [1]:
!pip install num2words
!pip install pymorphy2
!pip install natasha

     |████████████████████████████████| 102kB 2.5MB/s 
     |████████████████████████████████| 51kB 1.5MB/s 
     |████████████████████████████████| 7.1MB 7.3MB/s 
     |████████████████████████████████| 34.4MB 7.1MB/s 
     |████████████████████████████████| 51kB 4.1MB/s 
     |████████████████████████████████| 51kB 4.6MB/s 
  Created wheel for intervaltree: filename=intervaltree-3.0.2-cp36-none-any.whl size=25791 sha256=4ef5065fe2db19faf15c50c5316943a4ef1e220f40730001af5b3f1c78c51e2e
  Stored in directory: /root/.cache/pip/wheels/08/99/c0/5a5942f5b9567c59c14aac76f95a70bf11dccc71240b91ebf5
Successfully built intervaltree
  Found existing installation: intervaltree 2.1.0
    Uninstalling intervaltree-2.1.0:
      Successfully uninstalled intervaltree-2.1.0


In [2]:
import nltk
nltk.download()

NLTK Downloader
---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------
Downloader> d

Download which package (l=list; x=cancel)?
  Identifier> stopwords
      Unzipping corpora/stopwords.zip.

---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------
Downloader> q


True

In [3]:
import re

from bs4 import BeautifulSoup
import pymorphy2
from num2words import num2words

from nltk.corpus import stopwords


from natasha import (
    Segmenter,
    MorphVocab,
    
    NewsEmbedding,
    NewsMorphTagger,
    NewsNERTagger,

    NamesExtractor,
    AddrExtractor,
    Doc
)

import numpy as np

In [26]:
#My little collection of functions for preprocessing for a single text


def emoji_replacer(text,emoji_list,replacers):
  #Transform emoji into words

  for index,emoji in enumerate(emoji_list):
    text = text.replace(emoji,' '+ replacers[index] +' ')

  return text


def text_early_preproc(text,del_html = True,del_punct_sp_chars=True,
                 del_underscore=True, del_digits=False):
  #Clean the text from artifacts and punctuation

  #Delete whitespaces and special string symbols
  text = re.sub("^\s+|\n|\r|\s+$", ' ', text)

  #Delete html tags
  if del_html:
    soap = BeautifulSoup(text, 'html.parser')
    text = soap.get_text()

  #Delete punctuation and other artifacts
  if del_punct_sp_chars:
    text = re.sub(r'[^\w\s]','',text)

  #Delete '_'
  if del_underscore:
    text = text.replace('_','')

  #Delete digits
  if del_digits:
    text = re.sub(r'\d+', '', text)

                 
  return text


def lemmatize_lower_case(text):
  #lemmatizationa in lower case

  words = text.lower().split()
  morph = pymorphy2.MorphAnalyzer()

  normal_tokens= [morph.parse(word)[0].normal_form for word in words]

  return " ".join(normal_tokens)


def delete_stop_words(text):
  #delete Russian stopwords

  tokens = [token for token in text.split() if token not in stopwords.words("russian")]
  text = " ".join(tokens)
  return text

def numbers_to_text(text):
  #Converts numbers into Russian text

  tokens = text.split()
  text = " ".join([num2words(token,lang='ru') if token.isnumeric() else token for token in tokens ])
  
  return text


def delete_digits(text):
  #Delete digits
  text = re.sub(r'\d+', '', text)

  return text

In [27]:
#Functions to work with named entities

def ne_extraction(text,
                  segmenter,morph_vocab,
                  morph_tagger,ner_tagger,
                  del_names=False,del_addr=False):


  #Extract, normalize and delete (optional) named entities


  doc = Doc(text)

  doc.segment(segmenter)
  doc.tag_ner(ner_tagger)
  doc.tag_morph(morph_tagger)


  for span in doc.spans:
    span.normalize(morph_vocab)

  for span in doc.spans:

    if span.type == 'PER':
      span.extract_fact(names_extractor)

    if span.type == 'LOC':
      span.extract_fact(addr_extractor)

  if del_names:
    for span in doc.spans:
      if span.type == 'PER':
        text = text.replace(span.text,'')


  if del_addr:
    for span in doc.spans:
      if span.type == 'LOC':
        text = text.replace(span.text,'')

  normal_ne = {}
  normal_ne['NAMES'] = list(np.unique([span.normal for span in doc.spans if span.type == 'PER']))
  normal_ne['LOCATIONS'] = list(np.unique([span.normal for span in doc.spans if span.type == 'LOC']))

  return text, normal_ne


def add_normal_ne(text,normal_ne):
  #Add extracted and normalized named entities in the end

  names = ["_".join(ne.split()) for ne in normal_ne['NAMES']]

  locations = ["_".join(ne.split()) for ne in normal_ne['LOCATIONS']]

  return " ".join([text] + names + locations)

In [50]:
text = '''<div> 

<p> 

Очень хочу поздравить своего хорошего друга и учителя Александра Петровича Иванова, сегодня ему

50 лет11!!!:)
Великолепная дата, поэтому желаю ему, чтобы был здоров    и весел, и прожил еще 100500 лет!1!11!!|\\\\

Александру Петровичу Иванову респект!!

Короче, все это прекрасно, и теперь приступим к застолью, господа!!!

Я щас!!:))))

Жаль, что завтра на работу:((((

Конечно, люблю Саратов, но в Адлере сейчас лучше!!

Но в любом случае передаю привет моей Любови Ильиничне Кизляркиной из Магадана!!



</p> </div>

'''

In [51]:
text

'<div> \n\n<p> \n\nОчень хочу поздравить своего хорошего друга и учителя Александра Петровича Иванова, сегодня ему\n\n50 лет11!!!:)\nВеликолепная дата, поэтому желаю ему, чтобы был здоров    и весел, и прожил еще 100500 лет!1!11!!|\\\\\n\nАлександру Петровичу Иванову респект!!\n\nКороче, все это прекрасно, и теперь приступим к застолью, господа!!!\n\nЯ щас!!:))))\n\nЖаль, что завтра на работу:((((\n\nКонечно, люблю Саратов, но в Адлере сейчас лучше!!\n\nНо в любом случае передаю привет моей Любови Ильиничне Кизляркиной из Магадана!!\n\n\n\n</p> </div>\n\n'

In [39]:
#Initialise Natasha main tools

segmenter = Segmenter()
morph_vocab = MorphVocab()
emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)
ner_tagger = NewsNERTagger(emb)

names_extractor = NamesExtractor(morph_vocab)
addr_extractor = AddrExtractor(morph_vocab)

In [73]:
%%time

#Example

text_res,ne = ne_extraction(text,
                  segmenter,morph_vocab,
                  morph_tagger,ner_tagger,
                  del_names=True,del_addr=True)

emoji_list = [':)',':(']
replacers = ['happy','sad']
text_res = emoji_replacer(text_res,emoji_list,replacers)

text_res = text_early_preproc(text_res,del_html = True,del_punct_sp_chars=True,
                 del_underscore=True, del_digits=False)

text_res = numbers_to_text(text_res)

text_res = delete_digits(text_res)

text_res = lemmatize_lower_case(text_res)

text_res = delete_stop_words(text_res)

text_res = add_normal_ne(text_res,ne)



CPU times: user 172 ms, sys: 115 ms, total: 287 ms
Wall time: 170 ms


In [53]:
text_res

'очень хотеть поздравить свой хороший друг учитель сегодня пятьдесят год happy великолепный дата поэтому желать здоровый весёлый прожить ещё сто тысяча пятьсот год респект короче весь это прекрасно приступить застолье господин щас happy жаль завтра работа sad любить хороший любой случай передавать привет Александр_Петрович_Иванов Любовь_Ильинична_Кизляркина Адлер Магадан Саратов'

## For the purpose of preprocessing a dataframe with lots of texts, after exploring several examples, I use all selected and tuned functions in a loop for all the texts.

Altought the approach with many different fuctions would be convenient (we can easily debug and add/delete functionalities in our preprocessing loop), it wouldn't be the fastest algorithm, would it? In all this fuctions there are some duplicating operations (for example, split(), join()), and maybe many transfer operations aren't a good idea either. With this example, I want to roughly demonstrate the microservices and monoliths architectures comparison for a little prototype of an app. Now I will try to create some king of monolith function without described problems (but with other).

In [82]:
def text_preprocessing(text, segmenter,morph_vocab,
                      morph_tagger,ner_tagger,
                      emoji_list, replacers,                   
                      del_names=False,del_addr=False,
                      del_html = True,del_punct_sp_chars=True,
                      del_underscore=True):
  
  #Text preprocessing function


  #Extract, normalize and delete (optional) named entities

  doc = Doc(text)

  doc.segment(segmenter)
  doc.tag_ner(ner_tagger)
  doc.tag_morph(morph_tagger)


  for span in doc.spans:
    span.normalize(morph_vocab)

  for span in doc.spans:

    if span.type == 'PER':
      span.extract_fact(names_extractor)

    if span.type == 'LOC':
      span.extract_fact(addr_extractor)

  if del_names:
    for span in doc.spans:
      if span.type == 'PER':
        text = text.replace(span.text,'')


  if del_addr:
    for span in doc.spans:
      if span.type == 'LOC':
        text = text.replace(span.text,'')

  normal_ne = {}
  normal_ne['NAMES'] = list(np.unique([span.normal for span in doc.spans if span.type == 'PER']))
  normal_ne['LOCATIONS'] = list(np.unique([span.normal for span in doc.spans if span.type == 'LOC']))

  #Transform emoji into words

  for index,emoji in enumerate(emoji_list):
    text = text.replace(emoji,' '+ replacers[index] +' ')


  #Clean the text from artifacts and punctuation

  #Delete whitespaces and special string symbols
  text = re.sub("^\s+|\n|\r|\s+$", ' ', text)

  #Delete html tags
  if del_html:
    soap = BeautifulSoup(text, 'html.parser')
    text = soap.get_text()

  #Delete punctuation and other artifacts
  if del_punct_sp_chars:
    text = re.sub(r'[^\w\s]','',text)

  #Delete '_'
  if del_underscore:
    text = text.replace('_','')
                 

  #Converts numbers into Russian text,
  #lemmatize in lower case
  #delete Russian stopwords and digits
  # in a single loop  

  morph = pymorphy2.MorphAnalyzer()

  tokens = text.lower().split()

  new_tokens = []
  for token in tokens:
    
    if token.isnumeric():
      token = " ".join([morph.parse(word)[0].normal_form for word in num2words(token,lang='ru').split()])
    else:
      token = re.sub(r'\d+', '', token)
      token = morph.parse(token)[0].normal_form

    if token not in stopwords.words("russian"):
      new_tokens.append(token)
  
  #Add extracted and normalized named entities in the end

  names = ["_".join(ne.split()) for ne in normal_ne['NAMES']]

  locations = ["_".join(ne.split()) for ne in normal_ne['LOCATIONS']]

  return " ".join(new_tokens + names + locations)

In [86]:
%%time

#Another example

text_res = text_preprocessing(text, segmenter,morph_vocab,
                      morph_tagger,ner_tagger,
                      emoji_list, replacers,                   
                      del_names=True,del_addr=True,
                      del_html = True,del_punct_sp_chars=True,
                      del_underscore=True)


CPU times: user 167 ms, sys: 113 ms, total: 280 ms
Wall time: 153 ms


In [81]:
text_res

'очень хотеть поздравить свой хороший друг учитель сегодня пятьдесят год happy великолепный дата поэтому желать здоровый весёлый прожить ещё сто тысяча пятьсот год респект короче весь это прекрасно приступить застолье господин щас happy жаль завтра работа sad любить хороший любой случай передавать привет Александр_Петрович_Иванов Любовь_Ильинична_Кизляркина Адлер Магадан Саратов'

We got a code, that is hard to maintain, but it's slightly faster (of course, there can be some random causes, and this time difference is statistically insignificant) and shorter, when it comes to type the code. 

Personally, I would use the first version with many functions, because it is easier to tune.

## So, it was my little example of the monoliths and microservices comparison.